In [37]:
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
import re
from dtype_diet import report_on_dataframe, optimize_dtypes
import os
from dataprep.clean import clean_headers
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip'
r = requests.get(path, verify = False)
files = ZipFile(BytesIO(r.content))
pattern = re.compile("Porto Alegre", flags = re.IGNORECASE)
arquivo = []
for file in files.namelist():
    if re.search(pattern, str(file)):
        arquivo.append(file)
temp_poa = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", skiprows=8)

temp_poa_desc = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", nrows=7, header = None)
desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}

In [24]:
temp_poa.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2022/01/01,0000 UTC,0,"1000,3","1000,3","999,4",NaN,25,"19,7","26,4",25,"19,7","19,3",72.0,65.0,72.0,126.0,"5,4","2,1",NaN
1,2022/01/01,0100 UTC,0,"1000,6","1000,6","1000,2",NaN,"24,1","19,8",25,"24,1","19,8","19,6",77.0,72.0,77.0,95.0,"4,3","1,3",NaN
2,2022/01/01,0200 UTC,0,"1000,5","1000,8","1000,5",NaN,"23,6","19,9","24,1","23,6","19,9","19,8",80.0,77.0,80.0,76.0,"3,4",1,NaN
3,2022/01/01,0300 UTC,0,1000,"1000,5",1000,NaN,"23,3","19,6","23,8","23,3","19,9","19,5",80.0,78.0,79.0,104.0,"2,6",1,NaN
4,2022/01/01,0400 UTC,0,"999,7",1000,"999,6",NaN,23,"20,1","23,6","22,5","20,2","19,4",85.0,78.0,84.0,132.0,"3,4","1,4",NaN


In [23]:
temp_poa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8760 non-null   object 
 1   Hora UTC                                               8760 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   object 
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)            

In [25]:
proposed_df = report_on_dataframe(temp_poa, unit="MB")
new_df = optimize_dtypes(temp_poa, proposed_df)
print(f'Original df memory: {temp_poa.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 7.92751407623291 MB
Propsed df memory: 0.9679117202758789 MB


In [26]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype   
---  ------                                                 --------------  -----   
 0   Data                                                   8760 non-null   category
 1   Hora UTC                                               8760 non-null   category
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   category
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   category
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   category
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   category
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   category
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   category
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)  

In [27]:
desc

{'REGIAO:': 'S',
 'UF:': 'RS',
 'ESTACAO:': 'PORTO ALEGRE - JARDIM BOTANICO',
 'CODIGO (WMO):': 'A801',
 'LATITUDE:': '-30,05361111',
 'LONGITUDE:': '-51,17472221',
 'ALTITUDE:': '41,18'}

In [53]:
df = pd.DataFrame()
for ano in range(2000, 2024):
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = []
    for file in files.namelist():
        if file[-4:].lower() == ".csv":
            arquivos.append(file)
        else:
            continue
    df01 = pd.DataFrame()
    for arquivo in arquivos:
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
        df02["ESTACAO"] = desc['ESTACAO:']
        df02["UF"] = desc['UF:']
        df02["REGIAO"] = desc['REGIAO:']
        #nome_arquivo = arquivo.split('/')[1]
        #df02.to_parquet(f'../../../data/inmet/inmet_{nome_arquivo[:-4]}.parquet')
        df01 = pd.concat([df01, df02])
    df = pd.concat([df, df01])

IndexError: list index out of range

In [24]:
df = pd.DataFrame()
for ano in range(2000, 2024):
    print(ano)
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = [file for file in files.namelist() if file.lower().endswith(".csv")]

    df01 = pd.DataFrame()
    nomes_colunas = []
    for arquivo in arquivos:
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        info2 = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8, nrows=1)
        df02["ESTACAO"] = info2['ESTACAO:']
        df02["UF"] = info2['UF:']
        df02["REGIAO"] = info2['REGIAO:']
        df01 = pd.concat([df01, df02])
        for col in df02.columns.to_list():
            nomes_colunas.append(col)

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [38]:
df01.sample(5)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",ESTACAO,UF,REGIAO
0,2023/01/01,0000 UTC,0,925,925,"924,1",NaN,"22,3","20,5","23,1",...,"20,2",90.0,84.0,89.0,70.0,3,"1,7",PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,"6,4","1012,9",1013,"1012,3",0,"23,4","22,6","24,6",...,"21,8",95.0,86.0,95.0,352.0,"4,5",",4",PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,0,"968,7","968,7","967,7",NaN,"23,6","21,6","23,7",...,"21,5",90.0,88.0,89.0,328.0,3,",9",PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,0,"991,2","991,2","990,8",NaN,"24,7","22,8","25,2",...,"22,8",90.0,88.0,89.0,78.0,"2,3",",5",PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,0,"1014,5","1014,5","1013,9",NaN,"25,1","21,7","25,1",...,"21,3",82.0,80.0,82.0,55.0,"4,9","2,2",PORTO ALEGRE - JARDIM BOTANICO,RS,S


In [39]:
df01.nunique()

Data                                                       1
Hora UTC                                                   1
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                          17
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    362
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          367
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         370
RADIACAO GLOBAL (Kj/m²)                                   16
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             117
TEMPERATURA DO PONTO DE ORVALHO (°C)                     119
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               124
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               117
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         114
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         126
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                  60
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                  71
UMIDADE RELATIVA DO AR, HORARIA (%)                       69
VENTO, DIREÇÃO HORARIA (

In [40]:
for col in df01.columns:
    print(col)

Data
Hora UTC
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
RADIACAO GLOBAL (Kj/m²)
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)
TEMPERATURA DO PONTO DE ORVALHO (°C)
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)
UMIDADE RELATIVA DO AR, HORARIA (%)
VENTO, DIREÇÃO HORARIA (gr) (° (gr))
VENTO, RAJADA MAXIMA (m/s)
VENTO, VELOCIDADE HORARIA (m/s)
ESTACAO
UF
REGIAO


In [42]:
df01.drop("Unnamed: 19", axis=1, inplace=True)

In [56]:
colunas_ajustadas = clean_headers(df01).columns.to_list()
df01.columns = colunas_ajustadas

Column Headers Cleaning Report:
	22 values cleaned (100.0%)


In [59]:
df01.sample(3)

,data,hora_utc,precipitacao_total_horario_mm,pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b,pressao_atmosferica_max_na_hora_ant_aut_m_b,pressao_atmosferica_min_na_hora_ant_aut_m_b,radiacao_global_kj_m,temperatura_do_ar_bulbo_seco_horaria_c,temperatura_do_ponto_de_orvalho_c,temperatura_maxima_na_hora_ant_aut_c,...,temperatura_orvalho_min_na_hora_ant_aut_c,umidade_rel_max_na_hora_ant_aut_%,umidade_rel_min_na_hora_ant_aut_%,umidade_relativa_do_ar_horaria_%,vento_direcao_horaria_gr_gr,vento_rajada_maxima_m_s,vento_velocidade_horaria_m_s,estacao,uf,regiao
0,2023/01/01,0000 UTC,0,"1012,9","1012,9","1012,6",NaN,22,"18,8","22,3",...,"18,6",82.0,80.0,82.0,61.0,"4,9",3,PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,0,"1003,9","1003,9","1003,3",0,"23,3","22,4","23,4",...,"22,3",94.0,94.0,94.0,197.0,0,0,PORTO ALEGRE - JARDIM BOTANICO,RS,S
0,2023/01/01,0000 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PORTO ALEGRE - JARDIM BOTANICO,RS,S


# TODO: ajustar decimal (for coluna replace "," "." etc)

In [60]:
df01.dtypes

data                                                    object
hora_utc                                                object
precipitacao_total_horario_mm                           object
pressao_atmosferica_ao_nivel_da_estacao_horaria_m_b     object
pressao_atmosferica_max_na_hora_ant_aut_m_b             object
pressao_atmosferica_min_na_hora_ant_aut_m_b             object
radiacao_global_kj_m                                    object
temperatura_do_ar_bulbo_seco_horaria_c                  object
temperatura_do_ponto_de_orvalho_c                       object
temperatura_maxima_na_hora_ant_aut_c                    object
temperatura_minima_na_hora_ant_aut_c                    object
temperatura_orvalho_max_na_hora_ant_aut_c               object
temperatura_orvalho_min_na_hora_ant_aut_c               object
umidade_rel_max_na_hora_ant_aut_%                      float64
umidade_rel_min_na_hora_ant_aut_%                      float64
umidade_relativa_do_ar_horaria_%                       